Economic Diversification Exploratory Analysis notebook.
I will be utilizing the medallion architecture: **Bronze -> Silver -> Gold. **

Each layer will help narrow down the questions this EDA serves to answer.

**1) Bronze - Raw Inputs**
The Bronze layer preserves the original datasets with minimal transformation.
Its purpose is to establish trust in the data and clearly define the unit of analysis.


*   **Tax data:** State-level tax revenues by tax code and quarter,    capturing the legal and fiscal structure of state tax systems.
*   **Economic data:** Quarterly state-level economic indicators and GDP by industry, describing the composition and scale of each state’s economy.
*   **State Demographics Data:** Annual state-level demographic characteristics, including population totals, race and ethnicity counts, and selected migration measures.


***Narrative Question:***
"What data do we have, from which source, and at what granularity?"

---

In [6]:
# Imports for the Bronze layer & upload files
import pandas as pd

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Demographic_Dataset.csv to Demographic_Dataset.csv
Saving Economic_Dataset.csv to Economic_Dataset (1).csv
Saving Tax_Dataset.csv to Tax_Dataset (1).csv


In [9]:
#Load in the datasets
tax = pd.read_csv("Tax_Dataset.csv")
econ = pd.read_csv("Economic_Dataset.csv")
demo = pd.read_csv("Demographic_Dataset.csv")

#Make copies
tax_bronze = tax.copy()
econ_bronze = econ.copy()
demo_bronze = demo.copy()

In [10]:
# Light cleaning for Homogeniety and addition of join-safe column (period) to be used in silver and gold
tax_bronze["period"] = pd.PeriodIndex(year=tax_bronze["Year"], quarter=tax_bronze["Quarter"], freq="Q")
econ_bronze["period"] = pd.PeriodIndex(year=econ_bronze["Year"], quarter=econ_bronze["Quarter"], freq="Q")

tax_bronze["period"] = pd.PeriodIndex.from_fields(
    year=tax_bronze["Year"],
    quarter=tax_bronze["Quarter"],
    freq="Q"
)

econ_bronze["period"] = pd.PeriodIndex.from_fields(
    year=econ_bronze["Year"],
    quarter=econ_bronze["Quarter"],
    freq="Q"
)

/tmp/ipython-input-207436783.py:2: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  tax_bronze["period"] = pd.PeriodIndex(year=tax_bronze["Year"], quarter=tax_bronze["Quarter"], freq="Q")
/tmp/ipython-input-207436783.py:3: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  econ_bronze["period"] = pd.PeriodIndex(year=econ_bronze["Year"], quarter=econ_bronze["Quarter"], freq="Q")


((111800, 7),
 (4264, 43),
 (1054, 26),
    Year  Quarter    State                      Tax_Category Tax_Code  Amount  \
 0  2004        1  Alabama                      Property tax      T01  120297   
 1  2004        1  Alabama  General sales and gross receipts      T09  451925   
 2  2004        1  Alabama               Alcoholic beverages      T10   32628   
 
    period  
 0  2004Q1  
 1  2004Q1  
 2  2004Q1  ,
      State  Year  Quarter  Population  Unemployment_Rate  \
 0  Alabama  2005        1     4569805               4.90   
 1  Alabama  2005        2     4584599               4.50   
 2  Alabama  2005        3     4599393               4.23   
 
    Civilian noninstitutional population  Total Civilian Labor Force  \
 0                             3485077.0                   2144592.0   
 1                             3493724.0                   2147431.0   
 2                             3503938.0                   2155116.0   
 
    Total Employed Civilian Labor Force  Tota

**2) Silver - Analytical Structure** The Silver layer restructures the raw inputs into analysis-ready representations while preserving interpretability and comparability.

Key transformations include:


*   Aggregating tax revenues by state, quarter, and tax code, and computing tax-code shares to describe fiscal structure rather than scale.
*   Reshaping GDP data into industry-level components and computing GDP shares to describe economic composition.
*   Aligning annual demographic data to the quarterly timeline used in the tax and economic datasets, producing a consistent state-period demographic context.

***Narrative Question:***
"How are state tax systems, economies, and demographic contexts structured over time?"



---


In [11]:
# Build df tax_by_code : How much revenue does each tax code generate per state-quarter?
tax_by_code = (
    tax_bronze
    .groupby(["State", "period", "Tax_Code", "Tax_Category"], as_index=False)
    .agg(tax_revenue=("Amount", "sum"))
)

tax_by_code.head()

,State,period,Tax_Code,Tax_Category,tax_revenue
0,Alabama,2004Q1,T01,Property tax,120297
1,Alabama,2004Q1,T09,General sales and gross receipts,451925
2,Alabama,2004Q1,T10,Alcoholic beverages,32628
3,Alabama,2004Q1,T11,Amusements,19
4,Alabama,2004Q1,T12,Insurance,54784


In [14]:
# Sanity Checks - dupes , non-negative tax revenue, quick coverage test

#duplicates
dupes = tax_by_code.duplicated(subset=["State", "period", "Tax_Code"]).sum()
print("Duplicate State-period-Tax_Code rows:", dupes)

#revenue should be non-negative
print("Min tax_revenue:", tax_by_code["tax_revenue"].min())

#quick coverage check
print("Tax periods:", tax_by_code["period"].min(), "→", tax_by_code["period"].max())
print("Number of states:", tax_by_code["State"].nunique())
print("Number of tax codes:", tax_by_code["Tax_Code"].nunique())

Duplicate State-period-Tax_Code rows: 0
Min tax_revenue: -258488
Tax periods: 2004Q1 → 2025Q2
Number of states: 52
Number of tax codes: 26


Observation: negative tax revenue was found, indicating that there was either a refund, accounting adjustment, time correction, or retroactive policy effect.


---



In [18]:
# computing column - tax_code_shares: Within each state-quarter, how important is each tax code relative to the total tax revenue?

# compute total tax revenue per state-quarter
tax_totals = (
    tax_by_code
    .groupby(["State", "period"], as_index=False)
    .agg(total_tax_revenue=("tax_revenue", "sum"))
)

tax_totals.head()

# merge tax_totals into tax_by_code
tax_code_shares = tax_by_code.merge(
    tax_totals,
    on=["State", "period"],
    how="left"
)



# calculate tax shares (%)
tax_code_shares["tax_share"] = (
    tax_code_shares["tax_revenue"] / tax_code_shares["total_tax_revenue"]
)

tax_code_shares.head()

,State,period,Tax_Code,Tax_Category,tax_revenue,total_tax_revenue,tax_share
0,Alabama,2004Q1,T01,Property tax,120297,1661513,0.072402
1,Alabama,2004Q1,T09,General sales and gross receipts,451925,1661513,0.271996
2,Alabama,2004Q1,T10,Alcoholic beverages,32628,1661513,0.019638
3,Alabama,2004Q1,T11,Amusements,19,1661513,0.000011
4,Alabama,2004Q1,T12,Insurance,54784,1661513,0.032972


In [19]:
# Sanity checks - no double counting
(
    tax_code_shares
    .groupby(["State", "period"])["tax_share"]
    .sum()
    .round(6)
    .value_counts()
    .head()
)

,count
tax_share,
1.0,4472


In [31]:
# GDP structure : How is each state's GDP composed across industries in each quarter?

#identify columns to exclude (no double counting)
gdp_exclude = {
    "GDP_Total", # GDP_Private_Total + GDP_Gov_Total
    "GDP_Gov_Total", # GDP_Gov_Total = Federal + Military +State/Local
    "GDP_Private_Total", # Everything else combined industy
    "GDP_Manufacturing"  # parent of Durable + Nondurable
}

gdp_cols = [
    c for c in econ_bronze.columns
    if c.startswith("GDP_") and c not in gdp_exclude
]

gdp_cols

#len(gdp_cols), gdp_cols

#reshaping GDP to long format
gdp_long = (
    econ_bronze
    .melt(
        id_vars=["State", "period", "GDP_Total"],
        value_vars=gdp_cols,
        var_name="GDP_Component",
        value_name="gdp_value"
    )
)



#compute GDP share of total GDP
gdp_long["gdp_share"] = (
    gdp_long["gdp_value"] / gdp_long["GDP_Total"]
)

gdp_long.head()

,State,period,GDP_Total,GDP_Component,gdp_value,gdp_share
0,Alabama,2005Q1,155647.0,GDP_Accom_Food,3458.5,0.022220
1,Alabama,2005Q2,157994.9,GDP_Accom_Food,3473.9,0.021987
2,Alabama,2005Q3,158924.5,GDP_Accom_Food,3491.7,0.021971
3,Alabama,2005Q4,162142.8,GDP_Accom_Food,3598.6,0.022194
4,Alabama,2006Q1,163613.4,GDP_Accom_Food,3722.3,0.022751


In [33]:
# sanity check - no double counting
(
    gdp_long
    .groupby(["State", "period"])["gdp_share"]
    .sum()
    .round(6)
    .value_counts()
    .head()
)

,count
gdp_share,
1.000000,2418
0.999999,468
1.000001,463
0.999998,172
1.000002,165


Observation:
Aggregate GDP categories were excluded from component

---



3) Gold - Narrative & insight The Gold layer synthesizes outputs from the Silver layer into story-ready datasets designed for interpretation and visualization.

This data supports:



*   Measures of tax diversification and economic diversification at the state-quarter level.
*   Identification of dominant tax codes and dominant GDP sectors.

*   Comparative analysis of how fiscal structure aligns—or fails to align—with economic composition, interpreted within each state’s demographic context.

**NOTE: **Demographic variables are used to contextualize and stratify results rather than to define fiscal or economic outcomes directly. (long story short - proxy indicator)

***Narrative Question:*** "How do state tax structures relate to economic composition and diversification, and how do these relationships vary across different demographic contexts?"



---



In [47]:
# tax_structure_gold : How diversified is each state's tax system in each quarted, and which tax code dominates?
# using silver df - tax_code_shares, we compute tax concetration
# if you would like to know more, this is an index called  Herfindahl-Hirschman Index (HHI), its a standard economic measure of market concentration
# (or anything for that matter), and it makes time series data more interpritable (standardizes the thing across multiple time periods and states)
# the closer a number is to 1 the more diversified tax base, the closer to 0 the more concentrated.


# negative revenue can influence the index so we remidy that first - preserve silver as source of truth, modify gold for narrative
tax_code_shares_gold = tax_code_shares.copy()

tax_code_shares_gold["tax_share_adj"] = tax_code_shares_gold["tax_share"].clip(lower=0)

# calculate HHI
tax_hhi = (
    tax_code_shares_gold
    .assign(hhi_component=lambda df: df["tax_share_adj"] ** 2)
    .groupby(["State", "period"], as_index=False)
    .agg(
        tax_hhi=("hhi_component", "sum"),
        total_tax_revenue=("total_tax_revenue", "first")
    )
)

# establish index - (tax_diversification is the index, not tax_hhi)
tax_hhi["tax_diversification"] = 1 - tax_hhi["tax_hhi"]
#tax_hhi.head()

# lets count how many active tax code per state in each quarter
tax_code_count = (
    tax_code_shares
    .query("tax_revenue != 0")
    .groupby(["State", "period"], as_index=False)
    .agg(num_tax_codes=("Tax_Code", "nunique"))
)

#
dominant_tax = (
    tax_code_shares
    .sort_values("tax_share", ascending=False)
    .groupby(["State", "period"], as_index=False)
    .first()[["State", "period", "Tax_Code", "tax_share"]]
    .rename(columns={
        "Tax_Code": "dominant_tax_code",
        "tax_share": "dominant_tax_share"
    })
)

# Identify dominant tax codes
dominant_tax = (
    tax_code_shares
    .sort_values("tax_share", ascending=False)
    .groupby(["State", "period"], as_index=False)
    .first()[["State", "period", "Tax_Code", "tax_share"]]
    .rename(columns={
        "Tax_Code": "dominant_tax_code",
        "tax_share": "dominant_tax_share"
    })
)

# Finally assemble: tax_structure_gold dataframe
tax_structure_gold = (
    tax_hhi
    .merge(tax_code_count, on=["State", "period"], how="left")
    .merge(dominant_tax, on=["State", "period"], how="left")
)

tax_structure_gold.head()

,State,period,tax_hhi,total_tax_revenue,tax_diversification,num_tax_codes,dominant_tax_code,dominant_tax_share
0,Alabama,2004Q1,0.172821,1661513,0.827179,23,T09,0.271996
1,Alabama,2004Q2,0.220876,1864646,0.779124,22,T40,0.365800
2,Alabama,2004Q3,0.199733,1744070,0.800267,23,T40,0.321840
3,Alabama,2004Q4,0.190773,1873053,0.809227,23,T40,0.320337
4,Alabama,2005Q1,0.179422,1856112,0.820578,22,T40,0.289471


In [51]:
# adding extra additions (like top 3 or 5) to the tax_struture_gold without changing the original df
K = 3

tax_topk = (
    tax_code_shares
    .sort_values(["State", "period", "tax_share"], ascending=[True, True, False])
    .groupby(["State", "period"])
    .head(K)
    .assign(rank=lambda df: df.groupby(["State","period"]).cumcount() + 1)
)

# Pivot into columns like top_tax_code_1, top_tax_share_1, ...
tax_topk_wide = tax_topk.pivot_table(
    index=["State", "period"],
    columns="rank",
    values=["Tax_Code", "Tax_Category", "tax_share"],
    aggfunc="first"
)

tax_topk_wide.columns = [f"top_tax_{name.lower()}_{rank}" for name, rank in tax_topk_wide.columns]
tax_topk_wide = tax_topk_wide.reset_index()

tax_structure_gold_expanded = tax_structure_gold.merge(
    tax_topk_wide,
    on=["State", "period"],
    how="left"
)

tax_structure_gold_expanded.head()

,State,period,tax_hhi,total_tax_revenue,tax_diversification,num_tax_codes,dominant_tax_code,dominant_tax_share,top_tax_tax_category_1,top_tax_tax_category_2,top_tax_tax_category_3,top_tax_tax_code_1,top_tax_tax_code_2,top_tax_tax_code_3,top_tax_tax_share_1,top_tax_tax_share_2,top_tax_tax_share_3
0,Alabama,2004Q1,0.172821,1661513,0.827179,23,T09,0.271996,General sales and gross receipts,Individual income taxes,Public utilities,T09,T40,T15,0.271996,0.271807,0.093721
1,Alabama,2004Q2,0.220876,1864646,0.779124,22,T40,0.365800,Individual income taxes,General sales and gross receipts,Public utilities,T40,T09,T15,0.365800,0.262760,0.076688
2,Alabama,2004Q3,0.199733,1744070,0.800267,23,T40,0.321840,Individual income taxes,General sales and gross receipts,Public utilities,T40,T09,T15,0.321840,0.274203,0.090120
3,Alabama,2004Q4,0.190773,1873053,0.809227,23,T40,0.320337,Individual income taxes,General sales and gross receipts,Public utilities,T40,T09,T15,0.320337,0.260995,0.080519
4,Alabama,2005Q1,0.179422,1856112,0.820578,22,T40,0.289471,Individual income taxes,General sales and gross receipts,Public utilities,T40,T09,T15,0.289471,0.271415,0.086082


In [48]:
# sanity check -
tax_structure_gold.describe()

,tax_hhi,total_tax_revenue,tax_diversification,num_tax_codes,dominant_tax_share
count,4472.000000,4.472000e+03,4472.000000,4472.000000,4472.000000
mean,0.289676,9.419161e+06,0.710324,21.262746,0.442882
std,0.078694,3.526138e+07,0.078694,2.053646,0.103683
min,0.126619,5.447200e+04,0.137885,14.000000,0.212983
25%,0.234564,1.366823e+06,0.672832,20.000000,0.366999
50%,0.280467,2.805258e+06,0.719533,22.000000,0.423782
75%,0.327168,5.947497e+06,0.765436,23.000000,0.501999
max,0.862115,4.732161e+08,0.873381,25.000000,0.928071


In [49]:
# GDP_Diversification_structure: How diversified is each state's economy in each quarter, and which GDP sector dominates?
# # using silver df - gdp_long, we compute GDP concetration

gdp_hhi = (
    gdp_long
    .assign(hhi_component=lambda df: df["gdp_share"] ** 2)
    .groupby(["State", "period"], as_index=False)
    .agg(
        gdp_hhi=("hhi_component", "sum"),
        GDP_Total=("GDP_Total", "first")
    )
)

# establish index - (gdp_diversification is the index, not gdp_hhi)
gdp_hhi["gdp_diversification"] = 1 - gdp_hhi["gdp_hhi"]

#gdp_hhi.head()

# lets count how many sectors per state in each quarter
gdp_component_count = (
    gdp_long
    .query("gdp_value != 0")
    .groupby(["State", "period"], as_index=False)
    .agg(num_gdp_components=("GDP_Component", "nunique"))
)

# Identify dominant gdp sectors
dominant_gdp = (
    gdp_long
    .sort_values("gdp_share", ascending=False)
    .groupby(["State", "period"], as_index=False)
    .first()[["State", "period", "GDP_Component", "gdp_share"]]
    .rename(columns={
        "GDP_Component": "dominant_gdp_component",
        "gdp_share": "dominant_gdp_share"
    })
)

# Finally assemble : gdp_structure_gold df
gdp_structure_gold = (
    gdp_hhi
    .merge(gdp_component_count, on=["State", "period"], how="left")
    .merge(dominant_gdp, on=["State", "period"], how="left")
)

gdp_structure_gold.head()

,State,period,gdp_hhi,GDP_Total,gdp_diversification,num_gdp_components,dominant_gdp_component,dominant_gdp_share
0,Alabama,2005Q1,0.065733,155647.0,0.934267,23,GDP_Durable_Goods,0.106287
1,Alabama,2005Q2,0.065409,157994.9,0.934591,23,GDP_Durable_Goods,0.106140
2,Alabama,2005Q3,0.065014,158924.5,0.934986,23,GDP_Durable_Goods,0.106651
3,Alabama,2005Q4,0.064988,162142.8,0.935012,23,GDP_Durable_Goods,0.110891
4,Alabama,2006Q1,0.065176,163613.4,0.934824,23,GDP_Durable_Goods,0.112073


In [52]:
# adding extra additions (like top 3 or 5) to the gdp_struture_gold without changing the original df
K = 3

gdp_topk = (
    gdp_long
    .sort_values(["State", "period", "gdp_share"], ascending=[True, True, False])
    .groupby(["State", "period"])
    .head(K)
    .assign(rank=lambda df: df.groupby(["State","period"]).cumcount() + 1)
)

gdp_topk_wide = gdp_topk.pivot_table(
    index=["State", "period"],
    columns="rank",
    values=["GDP_Component", "gdp_share"],
    aggfunc="first"
)

gdp_topk_wide.columns = [f"top_gdp_{name.lower()}_{rank}" for name, rank in gdp_topk_wide.columns]
gdp_topk_wide = gdp_topk_wide.reset_index()

gdp_structure_gold_expanded = gdp_structure_gold.merge(
    gdp_topk_wide,
    on=["State", "period"],
    how="left"
)

gdp_structure_gold_expanded.head()

,State,period,gdp_hhi,GDP_Total,gdp_diversification,num_gdp_components,dominant_gdp_component,dominant_gdp_share,top_gdp_gdp_component_1,top_gdp_gdp_component_2,top_gdp_gdp_component_3,top_gdp_gdp_share_1,top_gdp_gdp_share_2,top_gdp_gdp_share_3
0,Alabama,2005Q1,0.065733,155647.0,0.934267,23,GDP_Durable_Goods,0.106287,GDP_Durable_Goods,GDP_Real_Estate,GDP_Gov_State_Local,0.106287,0.102710,0.100012
1,Alabama,2005Q2,0.065409,157994.9,0.934591,23,GDP_Durable_Goods,0.106140,GDP_Durable_Goods,GDP_Real_Estate,GDP_Gov_State_Local,0.106140,0.102973,0.099641
2,Alabama,2005Q3,0.065014,158924.5,0.934986,23,GDP_Durable_Goods,0.106651,GDP_Durable_Goods,GDP_Real_Estate,GDP_Gov_State_Local,0.106651,0.102590,0.099369
3,Alabama,2005Q4,0.064988,162142.8,0.935012,23,GDP_Durable_Goods,0.110891,GDP_Durable_Goods,GDP_Real_Estate,GDP_Gov_State_Local,0.110891,0.098952,0.098737
4,Alabama,2006Q1,0.065176,163613.4,0.934824,23,GDP_Durable_Goods,0.112073,GDP_Durable_Goods,GDP_Real_Estate,GDP_Gov_State_Local,0.112073,0.098774,0.097697


In [50]:
# sanity check -
gdp_structure_gold.describe()

,gdp_hhi,GDP_Total,gdp_diversification,num_gdp_components,dominant_gdp_share
count,4264.000000,4.264000e+03,4264.000000,4264.000000,4264.000000
mean,0.075164,7.375053e+05,0.924836,22.980769,0.146839
std,0.016574,2.723467e+06,0.016574,0.137351,0.040980
min,0.061032,2.354530e+04,0.812598,22.000000,0.093666
25%,0.066920,9.250652e+04,0.923964,23.000000,0.124017
50%,0.071190,2.333744e+05,0.928810,23.000000,0.137206
75%,0.076036,4.703413e+05,0.933080,23.000000,0.154896
max,0.187402,3.048573e+07,0.938968,23.000000,0.397987
